In [1]:
departure_airport = 'OVB'
destination_airport = 'MOW'
departure_date = '2019-12-31'

In [2]:
def extract_fresh_cookie(executable_path='geckodriver.exe'):
    '''
    Заходит на сайт аэрофлота, забирает свежий cookie bm_sz(необходим
    для осуществления поискового запроса)
    
    Args:
        executable_path (str): путь до бинарника geckodriver.exe (можно 
            взять на https://github.com/mozilla/geckodriver/releases/)
    
    Returns:
        bm_sz (str): куки, необходимый для дальнейшего поискового запроса
        
    '''
    import selenium.webdriver
    from selenium.webdriver.firefox.options import Options
    options = Options()
    options.headless = True
    browser = selenium.webdriver.Firefox(executable_path=executable_path, options=options)
    browser.get('https://www.aeroflot.ru/sb/app/ru-ru')
    cook = browser.get_cookie('bm_sz')
    bm_sz = cook['name']+'='+cook['value']
    browser.close()
    return(bm_sz)

def get_json_from_afl(departure = departure_airport, destination = destination_airport, departure_date = departure_date):
    '''
    Используя полученные от extract_fresh_cookie() куки, осуществляет поисковый запрос
    
    Args:
        departure     (str): код IATA города/аэропорта отправления
        destination   (str): код IATA города/аэропорта назначения
        depature_date (str): дата, на которую делается запрос (формат "ГГГГ-ММ-ДД")
    
    Returns:
        dict_data  (list(dict)): список словарей с необходимыми данными по рейсам
        num_of_flights    (int): количество выгруженных рейсов
        
    '''
    import requests
    params = {"routes":[{"origin":departure,"destination":destination,"departure_date":departure_date}],
              "cabin":"econom","award":'false',"country":"ru","adults":1,"children":0,"infants":0,"combined":'false',
              "coupon_codes":[],"lang":"ru","extra":{"extended":'false'},
              "client":{"ga_client_id":"","loyalty_id":"","loyalty_level":""}}
    sess = requests.Session()

    cookie = extract_fresh_cookie()

    headers = {'Host': 'www.aeroflot.ru',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:68.0) Gecko/20100101 Firefox/68.0',
            'Accept': 'application/json',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate, br',
            'Referer': 'https://www.aeroflot.ru/sb/app/ru-ru',
            'Content-Type': 'application/json',
            'Origin': 'https://www.aeroflot.ru',
            'Content-Length': '310',
            'Connection': 'keep-alive',
            'Cookie': cookie}

    t = sess.request('POST', 'https://www.aeroflot.ru/sb/booking/api/app/search/v5',
                     json=params, headers=headers)
    
    if t.status_code == 200:
        dict_data = t.json()['data']['itineraries'][0]
        num_of_flights = len(dict_data)
        print('Данные получены, количество рейсов - {}'.format(len(dict_data)))
        return(dict_data, num_of_flights)
    else:
        dict_data = None
        num_of_flights = 0
        print('При загрузке данных произошла ошибка, код {}'.format(t.status_code))
        return(dict_data, num_of_flights)
    
def json_to_pd(dict_data, num_of_flights):
    '''
    Используя полученные ранее данные от сайта aeroflot, конвертирует их
    в pd.Dataframe() в разрезе рейсов и ценовой политики
    
    Args:
        dict_data (list(dict)): список словарей с необходимыми данными по рейсам
        num_of_flights   (int): количество выгруженных рейсов
        
    Returns:
        flights (dataframe): датафрейм с данными по рейсам
        prices  (dataframe): датафрейм с данными по ценам
        
    '''
    global departure_date, departure_airport, destination_airport
    import pandas as pd
    import datetime        
        
    flights = pd.DataFrame()
    for flight in range(num_of_flights):
        flights_temp = pd.io.json.json_normalize(dict_data[flight]['legs'][0]['segments'])
        flights_temp['flight_index'] = datetime.datetime.now().strftime("%Y-%m-%d") + '_' + departure_date + '_' + str(flight)
        flights = pd.concat([flights, flights_temp])
    flights.set_index('flight_index')
    
    prices = pd.DataFrame()
    for flight in range(num_of_flights):
        prices_temp = pd.io.json.json_normalize(dict_data[flight]['prices'])
        prices_temp['flight_index'] = datetime.datetime.now().strftime("%Y-%m-%d") + '_' + departure_date + '_' + str(flight)
        prices = pd.concat([prices, prices_temp])
    prices.set_index('flight_index')
    
    return(flights, prices)

def column_type_to_str(dataframe):
    '''
    Приведение всех колонок таблицы с типом данных <list>
    к строковому типу
    (так как в sql базу нельзя вставить тип данных <list>)

    Args:
        dataframe (pd.Dataframe): датафрейм с данными

    Returns:
        dataframe (pd.Dataframe): датафрейм с данными
    '''
    columns_for_str = []
    for i in range(len(dataframe.columns)):
        if isinstance(dataframe.iloc[0,i], list):
            columns_for_str.append(dataframe.columns[i])
    for column in columns_for_str:
        dataframe[column] = dataframe[column].apply(lambda x: str(x))
    return(dataframe)
        
def pd_to_sqlite(flights, prices, database_name):
    '''
    Используя полученные датафреймы по рейсам и ценам, формирует БД sqlite
    
    Args:
        flights (pd.Dataframe): датафрейм с данными по рейсам
        prices  (pd.Dataframe): датафрейм с данными по ценам
        database_name    (str): имя базы данных в формате ".db" 
    
    '''
    import sqlite3
    flights = column_type_to_str(flights)
    prices = column_type_to_str(prices)
    
    conn = sqlite3.connect(database_name)
    cursor = conn.cursor()
    
    #Подготовка заголовков для таблицы flights
    flights_columns_string = '('
    flights_columns_string_placeholder = '('
    for column in flights.columns:
        flights_columns_string += '[{}] text, '.format(column)
        flights_columns_string_placeholder += '?,'
    flights_columns_string = flights_columns_string[:-2] + ')'
    flights_columns_string_placeholder = flights_columns_string_placeholder[:-1] + ')'
    
    #Подготовка заголовков для таблицы prices
    prices_columns_string = '('
    prices_columns_string_placeholder = '('
    for column in prices.columns:
        prices_columns_string += '[{}] text, '.format(column)
        prices_columns_string_placeholder += '?,'
    prices_columns_string = prices_columns_string[:-2] + ')'
    prices_columns_string_placeholder = prices_columns_string_placeholder[:-1] + ')'
    
    #Проверка наличия таблицы flights и последующее внесение данных
    try:
        cursor.execute("""CREATE TABLE flights
                          {}
                       """.format(flights_columns_string))
        print('В базе данных {} создана таблица flights'.format(database_name))
    except:
        print('Таблица flights уже есть в базе данных {}'.format(database_name))
    cursor.executemany("INSERT INTO flights VALUES {}".format(flights_columns_string_placeholder), flights.values)
    conn.commit()
    print('Данные по полетам внесены в базу данных {}'.format(database_name))
    
    #Проверка наличия таблицы prices и последующее внесение данных
    try:
        cursor.execute("""CREATE TABLE prices
                          {}
                       """.format(prices_columns_string))
        print('В базе данных {} создана таблица prices'.format(database_name))
    except:
        print('Таблица prices уже есть в базе данных {}'.format(database_name))
    cursor.executemany("INSERT INTO prices VALUES {}".format(prices_columns_string_placeholder), prices.values)
    conn.commit()
    print('Данные по ценам внесены в базу данных {}'.format(database_name))

In [3]:
dict_data, num_of_flights = get_json_from_afl(departure = departure_airport,
                                              destination = destination_airport,
                                              departure_date = departure_date)

flights, prices = json_to_pd(dict_data, num_of_flights)

pd_to_sqlite(flights, prices, 'aerflot_flights.db')

Данные получены, количество рейсов - 9
В базе данных aerflot_flights.db создана таблица flights
Данные по полетам внесены в базу данных aerflot_flights.db
В базе данных aerflot_flights.db создана таблица prices
Данные по ценам внесены в базу данных aerflot_flights.db
